<a href="https://colab.research.google.com/github/nebuchad-nezzar/Large-Language-Models/blob/main/Keyword_Based_Semantic_Search_Document_Retrieval.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
import math
from scipy import spatial


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
!pip install sentence_transformers openai unstructured

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 26.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.3/245.3 kB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.5/383.5 kB 28.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 57.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 26.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 553.2/553.2 kB 37.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 274.7/274.7 kB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.3/45.3 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.7/82.7 kB 7.7 MB/s eta 0:00:00
   ━━

Let's discuss why we need chunking/ need to divide data - context length

In [ ]:
txt = '''I would like to get your all  thoughts on the bond yield increase this week.  I am not worried about the market downturn but the sudden increase in yields. On 2/16 the 10 year bonds yields increased by almost  9 percent and on 2/19 the yield increased by almost 5 percent.

Key Points from the CNBC Article:

* **The “taper tantrum” in 2013 was a sudden spike in Treasury yields due to market panic after the Federal Reserve announced that it would begin tapering its quantitative easing program.**
* **Major central banks around the world have cut interest rates to historic lows and launched unprecedented quantities of asset purchases in a bid to shore up the economy throughout the pandemic.**
* **However, the recent rise in yields suggests that some investors are starting to anticipate a tightening of policy sooner than anticipated to accommodate a potential rise in inflation.**

The recent rise in bond yields and U.S. inflation expectations has some investors wary that a repeat of the 2013 “taper tantrum” could be on the horizon.

The benchmark U.S. 10-year Treasury note climbed above 1.3% for the first time since February 2020 earlier this week, while the 30-year bond also hit its highest level for a year. Yields move inversely to bond prices.

Yields tend to rise in lockstep with inflation expectations, which have reached their highest levels in a decade in the U.S., powered by increased prospects of a large fiscal stimulus package, progress on vaccine rollouts and pent-up consumer demand.

The “taper tantrum” in 2013 was a sudden spike in Treasury yields due to market panic after the Federal Reserve announced that it would begin tapering its quantitative easing program.

Major central banks around the world have cut interest rates to historic lows and launched unprecedented quantities of asset purchases in a bid to shore up the economy throughout the pandemic. The Fed and others have maintained supportive tones in recent policy meetings, vowing to keep financial conditions loose as the global economy looks to emerge from the Covid-19 pandemic.

However, the recent rise in yields suggests that some investors are starting to anticipate a tightening of policy sooner than anticipated to accommodate a potential rise in inflation.

With central bank support removed, bonds usually fall in price which sends yields higher. This can also spill over into stock markets as higher interest rates means more debt servicing for firms, causing traders to reassess the investing environment.

“The supportive stance from policymakers will likely remain in place until the vaccines have paved a way to some return to normality,” said Shane Balkham, chief investment officer at Beaufort Investment, in a research note this week.

“However, there will be a risk of another ‘taper tantrum’ similar to the one we witnessed in 2013, and this is our main focus for 2021,” Balkham projected, should policymakers begin to unwind this stimulus.

Long-term bond yields in Japan and Europe followed U.S. Treasurys higher toward the end of the week as bondholders shifted their portfolios.

“The fear is that these assets are priced to perfection when the ECB and Fed might eventually taper,” said Sebastien Galy, senior macro strategist at Nordea Asset Management, in a research note entitled “Little taper tantrum.”

“The odds of tapering are helped in the United States by better retail sales after four months of disappointment and the expectation of large issuance from the $1.9 trillion fiscal package.”

Galy suggested the Fed would likely extend the duration on its asset purchases, moderating the upward momentum in inflation.

“Equity markets have reacted negatively to higher yield as it offers an alternative to the dividend yield and a higher discount to long-term cash flows, making them focus more on medium-term growth such as cyclicals” he said. Cyclicals are stocks whose performance tends to align with economic cycles.

Galy expects this process to be more marked in the second half of the year when economic growth picks up, increasing the potential for tapering.

## Tapering in the U.S., but not Europe

Allianz CEO Oliver Bäte told CNBC on Friday that there was a geographical divergence in how the German insurer is thinking about the prospect of interest rate hikes.

“One is Europe, where we continue to have financial repression, where the ECB continues to buy up to the max in order to minimize spreads between the north and the south — the strong balance sheets and the weak ones — and at some point somebody will have to pay the price for that, but in the short term I don’t see any spike in interest rates,” Bäte said, adding that the situation is different stateside.

“Because of the massive programs that have happened, the stimulus that is happening, the dollar being the world’s reserve currency, there is clearly a trend to stoke inflation and it is going to come. Again, I don’t know when and how, but the interest rates have been steepening and they should be steepening further.”

## Rising yields a ‘normal feature’

However, not all analysts are convinced that the rise in bond yields is material for markets. In a note Friday, Barclays Head of European Equity Strategy Emmanuel Cau suggested that rising bond yields were overdue, as they had been lagging the improving macroeconomic outlook for the second half of 2021, and said they were a “normal feature” of economic recovery.

“With the key drivers of inflation pointing up, the prospect of even more fiscal stimulus in the U.S. and pent up demand propelled by high excess savings, it seems right for bond yields to catch-up with other more advanced reflation trades,” Cau said, adding that central banks remain “firmly on hold” given the balance of risks.

He argued that the steepening yield curve is “typical at the early stages of the cycle,” and that so long as vaccine rollouts are successful, growth continues to tick upward and central banks remain cautious, reflationary moves across asset classes look “justified” and equities should be able to withstand higher rates.

“Of course, after the strong move of the last few weeks, equities could mark a pause as many sectors that have rallied with yields look overbought, like commodities and banks,” Cau said.

“But at this stage, we think rising yields are more a confirmation of the equity bull market than a threat, so dips should continue to be bought'''

In [ ]:
len(txt)

6422

In [ ]:
from transformers import BertTokenizer, BertModel


tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [ ]:
# text = "This is a long text that needs to be encoded and decoded."
encoded_text = tokenizer.encode(txt, padding=True, truncation=True, return_tensors='pt')

In [ ]:
encoded_text.shape

torch.Size([1, 512])

In [ ]:
decoded_text = tokenizer.decode(encoded_text[0], skip_special_tokens=True)

In [ ]:
decoded_text

'i would like to get your all thoughts on the bond yield increase this week. i am not worried about the market downturn but the sudden increase in yields. on 2 / 16 the 10 year bonds yields increased by almost 9 percent and on 2 / 19 the yield increased by almost 5 percent. key points from the cnbc article : * * * the “ taper tantrum ” in 2013 was a sudden spike in treasury yields due to market panic after the federal reserve announced that it would begin tapering its quantitative easing program. * * * * * major central banks around the world have cut interest rates to historic lows and launched unprecedented quantities of asset purchases in a bid to shore up the economy throughout the pandemic. * * * * * however, the recent rise in yields suggests that some investors are starting to anticipate a tightening of policy sooner than anticipated to accommodate a potential rise in inflation. * * the recent rise in bond yields and u. s. inflation expectations has some investors wary that a re

In [ ]:
len(decoded_text)

2542

Let's Keyword Search using TF-IDF approach


# TF-IDF: A Simple Explanation

## What is TF-IDF?

TF-IDF stands for "Term Frequency-Inverse Document Frequency." It's a way to figure out how important a word is in a document or a collection of documents.

## Breaking it Down

### TF (Term Frequency)

- This measures how often a word appears in a document.
- The more a word appears, the more important it might be.

**Example:**
In the sentence "The cat sat on the mat," the word "the" appears twice, while "cat" appears once.

### IDF (Inverse Document Frequency)

- This measures how common or rare a word is across all documents.
- Rare words are often more important or informative.

**Example:**
If you have 1000 documents about animals, and "cat" appears in 100 of them, while "aardvark" appears in only 5, "aardvark" would have a higher IDF.

## Putting TF and IDF Together

TF-IDF combines these two measures:
- It gives a high score to words that appear frequently in a document but are rare across all documents.
- It gives a low score to words that appear in many documents or appear rarely in a document.

## Real-World Example

Imagine you have a collection of news articles:

1. An article about a football match might have high TF-IDF scores for words like "goal," "referee," and player names.
2. An article about cooking might have high TF-IDF scores for words like "recipe," "ingredients," and specific dish names.

Common words like "the," "and," or "is" would have low TF-IDF scores in both articles because they appear frequently in almost all documents.

## Why is TF-IDF Useful?

1. **Search Engines**: It helps rank search results by identifying the most relevant documents.
2. **Content Recommendation**: It can find similar articles or documents.
3. **Text Summarization**: It can identify the most important words or sentences in a document.
4. **Spam Detection**: It can help identify unusual word patterns that might indicate spam.

## Conclusion

TF-IDF is a simple but powerful tool that helps us understand the importance of words in documents. It balances how often a word appears in a document with how unique it is across all documents, giving us valuable insights into text data.

In [ ]:

# List of documents
documents = [
    "The cat is playing in the garden",
    "A dog and cat are good pets",
    "Cats love to chase mice",
    "machine learning is based on algorithms ",
    "Deep learning uses neural networks",
    "Recurrent networks have connections",
    "The cost of this shirt is $15"
]

# Tokenize documents into words

tokenized_docs = [word_tokenize(d) for d in documents]

# Inverted index mapping words to document IDs
index = {}
for i, doc in enumerate(tokenized_docs):
    for word in doc:
        if word not in index:
            index[word] = []
        index[word].append(i)

print(f'Inverted Index:\n {index}')



Inverted Index:
 {'The': [0, 6], 'cat': [0, 1], 'is': [0, 3, 6], 'playing': [0], 'in': [0], 'the': [0], 'garden': [0], 'A': [1], 'dog': [1], 'and': [1], 'are': [1], 'good': [1], 'pets': [1], 'Cats': [2], 'love': [2], 'to': [2], 'chase': [2], 'mice': [2], 'machine': [3], 'learning': [3, 4], 'based': [3], 'on': [3], 'algorithms': [3], 'Deep': [4], 'uses': [4], 'neural': [4], 'networks': [4, 5], 'Recurrent': [5], 'have': [5], 'connections': [5], 'cost': [6], 'of': [6], 'this': [6], 'shirt': [6], '$': [6], '15': [6]}


In [ ]:

# Compute TF-IDF weights for words
def tfidf(word, id):
    # Term frequency - no. of times word appears in document
    tf = tokenized_docs[id].count(word)

    # No. of documents containing the word
    df = len(index[word]) if word in index else 0

    # Inverse document frequency
    idf =  math.log(len(documents) / (df+1))

    return tf * idf



In [ ]:
# Search query
query = "25"
query_words = word_tokenize(query)

In [ ]:
scores = {id:0 for id in range(len(documents))}
for q in query_words:
    if q in index:
        for id in index[q]:
            scores[id] += tfidf(q, id)

scores

{0: 0, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0}

In [ ]:

# Rank documents
ranked = sorted(scores.items(), key=lambda x: x[1], reverse=True)

# Return ranked documents
for id, score in ranked:
    print(f"Document {id}: {documents[id]}")
    print(f"Score: {score}")

Document 1: A dog and cat are good pets
Score: 2.1000608288825715
Document 0: The cat is playing in the garden
Score: 0.8472978603872037
Document 2: Cats love to chase mice
Score: 0
Document 3: machine learning is based on algorithms 
Score: 0
Document 4: Deep learning uses neural networks
Score: 0
Document 5: Recurrent networks have connections
Score: 0
Document 6: The cost of this shirt is $15
Score: 0


Semantic Search

In [ ]:
from sentence_transformers import SentenceTransformer
import scipy.spatial
import pandas as pd


/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [ ]:
documents = [
    "The cat is playing in the garden",
    "A dog and cat are good pets",
    "Cats love to chase mice",
    "Machine learning is based on algorithms",
    "Deep learning uses neural networks",
    "Recurrent networks have connections",
     "The cost of this shirt is $15"
]


In [ ]:
model = SentenceTransformer('all-MiniLM-L6-v2').to('cuda') #cpu


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
print(model)

SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
)


In [ ]:
embeddings = model.encode(documents,convert_to_tensor=True).to('cpu')

In [ ]:
embeddings.shape

torch.Size([7, 384])

In [ ]:
embeddings

tensor([[ 0.0985, -0.0455,  0.0097,  ...,  0.0113,  0.0335, -0.0079],
        [ 0.0143, -0.0200,  0.0689,  ...,  0.0864,  0.0669,  0.0567],
        [ 0.0305, -0.0297,  0.0800,  ...,  0.1213,  0.0618,  0.0443],
        ...,
        [-0.1011, -0.0130,  0.0732,  ...,  0.0614,  0.0381, -0.0397],
        [-0.0761, -0.0688,  0.0270,  ...,  0.1124,  0.0479, -0.0509],
        [-0.0226,  0.0855,  0.0442,  ..., -0.2102, -0.0619,  0.0028]])

In [ ]:
# User query
query = "shirt that price 25 dollars"

# Encode query
query_embedding = model.encode(query)

In [ ]:
query_embedding.shape

(384,)

In [ ]:
scores = [spatial.distance.cosine(query_embedding, doc) for doc in embeddings]


In [ ]:
scores

[1.0835138037800789,
 1.0224751122295856,
 0.970310416072607,
 1.0289253275841475,
 0.944504801183939,
 1.0090155790452833,
 0.248891547256203]

In [ ]:
scores = [scipy.spatial.distance.cosine(query_embedding, doc) for doc in embeddings]

# Print each score
for i, doc in enumerate(documents):
    print(f"Document {i}: {documents[i]}")
    print(f"Score: {1 - scores[i]:.4f}")

print("Most similar document:", documents[scores.index(min(scores))])

Document 0: The cat is playing in the garden
Score: -0.0835
Document 1: A dog and cat are good pets
Score: -0.0225
Document 2: Cats love to chase mice
Score: 0.0297
Document 3: Machine learning is based on algorithms
Score: -0.0289
Document 4: Deep learning uses neural networks
Score: 0.0555
Document 5: Recurrent networks have connections
Score: -0.0090
Document 6: The cost of this shirt is $15
Score: 0.7511
Most similar document: The cost of this shirt is $15


In [ ]:
def semantic_search(query,embeddings, documents):
    query_embedding = model.encode(query)

    document_embeddings = embeddings
    scores = [scipy.spatial.distance.cosine(query_embedding, doc) for doc in document_embeddings]

    # Print scores
    for i, doc in enumerate(documents):
        print(f"{doc}: {1 - scores[i]:.4f}")

    most_similar_doc = documents[scores.index(min(scores))]
    print("Most similar document:", most_similar_doc)


In [ ]:
semantic_search("clothing that price 25 dollars",embeddings,documents)

The cat is playing in the garden: -0.0704
A dog and cat are good pets: 0.0587
Cats love to chase mice: 0.0460
Machine learning is based on algorithms: -0.0501
Deep learning uses neural networks: 0.0387
Recurrent networks have connections: 0.0367
The cost of this shirt is $15: 0.6509
Most similar document: The cost of this shirt is $15


In [ ]:
semantic_search("furry animal",embeddings,documents)

The cat is playing in the garden: 0.1846
A dog and cat are good pets: 0.4635
Cats love to chase mice: 0.3653
Machine learning is based on algorithms: 0.0034
Deep learning uses neural networks: 0.0457
Recurrent networks have connections: 0.0592
The cost of this shirt is $15: 0.0665
Most similar document: A dog and cat are good pets
